In [2]:
import glob
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [13]:
tweet_subject = 'kanyewest'

all_files = glob.glob(os.path.join(f'../data/raw/{tweet_subject}', "*.csv"))

In [14]:
data = pd.read_csv(all_files[0])
for i in all_files[1:]:
    tmp = pd.read_csv(i)
    data = pd.concat([data, tmp])
data.head()

,Unnamed: 0,id_str,created_at,retweets,description,text,screen_name,user_created_at,followers
0,0,1295527477440401408,Tue Aug 18 01:06:35 +0000 2020,0,fuck EU |⠀if you have pronouns in your bio a...,RT @kanyewest: Daddy daughter time ⛷ https://t...,SmileonYT,Sun Apr 12 17:01:39 +0000 2020,13
1,0,1295527478778195968,Tue Aug 18 01:06:35 +0000 2020,0,Executive | Social Robotics | Feminism | Anima...,RT @democratyfriend: @ProjectLincoln @kanyewes...,MGeoch,Sun Apr 28 13:20:28 +0000 2019,531
2,0,1295527490799185926,Tue Aug 18 01:06:38 +0000 2020,0,NaN,@RoyBatt42867599 @jonmillrr @kanyewest ik it s...,BillGatesJr4,Thu Nov 19 00:33:32 +0000 2015,1
3,0,1295527495278579712,Tue Aug 18 01:06:39 +0000 2020,0,Executive | Social Robotics | Feminism | Anima...,RT @PRodSpeaks: @kanyewest https://t.co/4UQhdS...,MGeoch,Sun Apr 28 13:20:28 +0000 2019,531
4,0,1295527498093142017,Tue Aug 18 01:06:39 +0000 2020,0,A man of many talents. Be Happy! Clients inclu...,RT @2chainz: Well I guess the album ain’t done,LordBagheera,Thu May 21 06:17:19 +0000 2009,3755


In [15]:
documents = data['description'].astype(str).to_list()
print(documents)

['fuck EU   |⠀if you have pronouns in your bio all your opinions are invalid 🇨🇿🇨🇿🇨🇿', 'Executive | Social Robotics | Feminism | Animal activism | Behavioural 🧬 | political and global current affairs for all', 'nan', 'Executive | Social Robotics | Feminism | Animal activism | Behavioural 🧬 | political and global current affairs for all', 'A man of many talents. Be Happy! Clients include: WIZ KHALIFA, 50 CENT, PUSHA-T, JAE MILLZ, GUCCI MANE, LIL WAYNE, & MORE.', '⭐️⭐️⭐️', 'what nigga ?!? I’m like.....extravagant🦋\U0001f90d✨. •♊️🔞🇵🇷✨ •snap: litty.nyahh', 'I love my family, I know my Lord... Tree Climbing is my way of living, martial arts is my way of life', '17. With The Stars In The Skies 🌌💘', 'Executive | Social Robotics | Feminism | Animal activism | Behavioural 🧬 | political and global current affairs for all', 'Big juice energy', "I'm a Mouthy American 🇺🇸 Commentator.\n\nProudly CENSORED by @Twitter & @jack 🤐\n\nFollow me on Parler: https://parler.com/profile/Politicalincorrect33/pos

In [16]:
no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [17]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [18]:
no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA

lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [19]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 3
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0
nan 𝘬𝘱𝘰𝘱 football
Topic 1
executive social robotics
Topic 2
blacklivesmatter sixburgh dubnation
Topic 3
world blm getting
Topic 4
love god way
Topic 5
mediawar cancelgoogle keepoann
Topic 6
ig sc follow
Topic 7
fuck probably hate
Topic 8
account fan heart
Topic 9
follow anti taylor
Topic 0
soon matter lives
Topic 1
love just fan
Topic 2
world blm don
Topic 3
maga trump jesus
Topic 4
life balls man
Topic 5
com https http
Topic 6
animal social political
Topic 7
truth cancelgoogle mediawar
Topic 8
nan music player
Topic 9
fuck like nicki
